In [2]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase
import os
import langchain_core

In [3]:
from Flemmings_Library.variables import db_path, db
from Flemmings_Library.ollama import get_ollama_model_names

!ollama list > nul 2>&1 # Für Windows: Ollama starten, Ausgabe der Modelle unterdrücken

In [4]:
model_names = get_ollama_model_names(debug=False)
db_dialect = db.dialect
model_name = model_names[0] # Modell festlegen

In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
)
print(f'{model_name} ausgewählt und geladen.')

llama3.2-vision:11b ausgewählt und geladen.


Schema der Datenbank extrahieren

In [6]:
from Flemmings_Library.extract import extract_schema

schema = extract_schema(db_path, sample_rows=1, verbose=False)
schema_info_str = str(schema)

State-Klasse definieren

In [7]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict

class State(TypedDict):
    schema_info: str
    prompt: str
    question: str      
    db_dialect: str
    query: str           
    execution_result: str
    answer: str
    error: str

Prompt-Template erstellen und State initiieren

In [24]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

In [43]:
question = "List all Artists that start with the letter 'A'."

In [44]:
from Flemmings_Library.variables import chat_prompt_template_sql

In [45]:
formatted_prompt = chat_prompt_template_sql.format(
    schema_info=schema_info_str,
    question=question,
    db_dialect=db_dialect
)
print(formatted_prompt)


System: You are a helpful SQL assistant that provides only SELECT-Statements. Based on the following database schema, translate a natural language query into an SQL query that is executable in my sqlite-Database.

Database Schema:
{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embrae

In [46]:
# initialen State festlegen
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db.dialect,
    "prompt": formatted_prompt,
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

Query durch LLM ausgeben lassen und ausführen

In [47]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt_template_sql | llm | StrOutputParser()

In [48]:
try:
    llm_output = chain.invoke({
        "schema_info": schema_info_str,
        "question": question,
        "db_dialect": db.dialect
    })

    state["query"] = llm_output
    state['execution_result'] = db.run(state['query'])
    state['error'] = None

except Exception as e:
    state['error'] = str(e)
    state['execution_result'] = None


In [49]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe

Error-Handler-LLM, falls es zuvor einen Error gab

In [37]:
from Flemmings_Library.variables import prompt_sql_fix_template

In [38]:
formatted_prompt_sql_fix = prompt_sql_fix_template.format(
    schema_info=state["schema_info"],
    question=state["question"],
    query=state["query"],
    error=state["error"],
    db_dialect=state["db_dialect"]
)

print(formatted_prompt_sql_fix)

System: You are a helpful SQL assistant and you will help me to fix my SQL query for my sqlite-Database. I will show you the database schema, the question and the error that I got.
Please only answer based on the provided information.

Human: User Question: What City is the Customer with the highest ID from?
SQL Query: SELECT T2.City FROM Customer AS T1 INNER JOIN Address AS T2 ON T1.AddressId = T2.AddressId WHERE T1.CustomerId = ( SELECT MAX(CustomerId) FROM Customer )
Error: (sqlite3.OperationalError) no such table: Address
[SQL: SELECT T2.City FROM Customer AS T1 INNER JOIN Address AS T2 ON T1.AddressId = T2.AddressId WHERE T1.CustomerId = ( SELECT MAX(CustomerId) FROM Customer )]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Database-Schema: {'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': 

In [39]:
chain_sql_fix = prompt_sql_fix_template | llm | StrOutputParser()

In [40]:
llm_answer = chain_sql_fix.invoke({
            "db_dialect": state["db_dialect"],
            "schema_info": state["schema_info"],
            "question": state["question"],
            "query": state["query"],
            "error": state["error"]
        })

state["answer"] = llm_answer

In [41]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe

Antwort des Modells in Textform

In [50]:
from Flemmings_Library.variables import answer_prompt_template

In [51]:
formatted_answer_prompt = answer_prompt_template.format(
    question=state["question"],
    query=state["query"],
    execution_result=state["execution_result"]
)

print(formatted_answer_prompt)

System: You are a helpful SQL assistant. Based on the SQL query and its execution result, provide a concise final answer to the user's question.
Human: User Question: List all Artists that start with the letter 'A'.
SQL Query: SELECT Name FROM Artist WHERE Name LIKE 'A%'
Execution Result: [('AC/DC',), ('Accept',), ('Aerosmith',), ('Alanis Morissette',), ('Alice In Chains',), ('Antônio Carlos Jobim',), ('Apocalyptica',), ('Audioslave',), ('Azymuth',), ('A Cor Do Som',), ('Aquaman',), ("Aerosmith & Sierra Leone's Refugee Allstars",), ('Avril Lavigne',), ('Aisha Duo',), ('Aaron Goldberg',), ('Alberto Turco & Nova Schola Gregoriana',), ('Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker',), ('Academy of St. Martin in the Fields & Sir Neville Marriner',), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner',), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair',), ('Aaron Copland & London Symphony Orchestra',), ('A

In [52]:
chain_answer = answer_prompt_template | llm | StrOutputParser()

In [53]:
llm_answer = chain_answer.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "query": state["query"],
            "execution_result": state["execution_result"]
        })

state["answer"] = llm_answer

In [54]:
state

{'schema_info': "{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)]}, 'Employe